In [ ]:
import pandas as pd
import numpy as np

from sqlalchemy import create_engine
engine = create_engine('postgresql://****') #removed

In [ ]:
query = '''
select 
d.id as donation_id,
amount_in_cents,
codonation_amount_in_cents,
category as donation_type, 
receiver_name as project_name, 
c.name as main_category
from donations d 
inner join 
projects p
on d.receiver_identifier = p.identifier 
inner join
main_categories mc
on mc.project_id = p.project_id
inner join
categories c
on c.id = mc.category_id
where category = 'UserDonation';
'''

test_data = pd.read_sql(sql=query, con=engine)
test_data.head()

# Predicting the donation amount by category

In [ ]:
y = test_data['amount_in_cents']
y.shape

In [ ]:
X = pd.get_dummies(test_data['main_category'], prefix='mc')
X.shape

In [ ]:
X.head()

## Statistics

In [ ]:
def cents_to_eur(val):
    return round(val / 100, 0)

def plot_stats_per_category(value_column):
    df = test_data[['main_category', value_column]]
    df.columns = ['main_category', 'amount in EUR']
    grp = df.groupby('main_category')
    stats = pd.DataFrame(grp.describe().apply(cents_to_eur).astype(int))
    plot = stats['amount in EUR'][['count', 'mean', 'std', 'min', 'max']].plot(kind='bar', subplots=True, figsize=(6, 9)); plt.legend(loc='best')
    return plot, stats

plot, stats = plot_stats_per_category('amount_in_cents')

In [ ]:
stats

## Prediction

In [ ]:
from sklearn.cross_validation import train_test_split
from sklearn import grid_search
from sklearn.tree import DecisionTreeRegressor

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from sklearn import linear_model

def train_linear_model(X,y):
    model = linear_model.LinearRegression()
    model.fit(X, y)
    print('Coefficients: \n', model.coef_)
    return model

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
model = train_linear_model(X_train, y_train)

In [ ]:
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit

def plot_learning_curve(estimator, title, X, y, ylim=None):
    '''
    See http://scikit-learn.org/stable/auto_examples/model_selection/plot_learning_curve.html#sphx-glr-auto-examples-model-selection-plot-learning-curve-py
    '''
    plt.figure()
    plt.title(title)
    
    if ylim is not None:
        plt.ylim(*ylim)

    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(estimator, X, y, n_jobs=1)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt


plt = plot_learning_curve(model, 'Learning curve', X, y)

plt.show()

In [ ]:
def predict_for_main_categories(X, model, spendenart):
    predictions = {}
    dummies_table = X.head(0)

    for category in test_data['main_category'].unique():
        dummies_table.loc[0] = np.zeros(24)
        dummies_table['mc_' + category][0] = 1
        value = model.predict(dummies_table)
        predictions[category] = value


    pred_df = pd.DataFrame(predictions).transpose()
    donation_results = pred_df.reset_index(inplace=False)
    donation_results.columns = 'Kategorie', spendenart
    donation_results[spendenart] = donation_results[spendenart] / 100
    donation_results.sort_values(by=spendenart, inplace=True)
    donation_results[spendenart] = donation_results[spendenart].astype(int)
    return donation_results

donations = predict_for_main_categories(X, model, 'Spende in Euro')

In [ ]:
donations

In [ ]:
#donations.to_csv('vorhergesagte_spendenhoehe_pro_hauptkategorie.csv', index=False, sep=';')

# Predicting the codonation by category

In [ ]:
y_codonation = test_data['codonation_amount_in_cents']
y_codonation.shape

## Statistiken

In [ ]:
codonation_plot, codonation_stats = plot_stats_per_category('codonation_amount_in_cents')

In [ ]:
codonation_stats

In [ ]:
X_train_cd, X_test_cd, y_train_cd, y_test_cd = train_test_split(X, y_codonation, test_size=0.3)
model_codonation = train_linear_model(X_train_cd, y_train_cd)

In [ ]:
plt = plot_learning_curve(model, 'Learning curve', X, y_codonation)

plt.show()

In [ ]:
codonations = predict_for_main_categories(X, model_codonation, 'Mitspende in Euro')
codonations

In [ ]:
#codonations.to_csv('vorhergesagte_mitspendenhoehe_pro_hauptkategorie.csv', index=False, sep=';')